In [0]:
#imports 
#from tensorflow import keras
import keras
import pandas as pd
import numpy as np
from keras.layers import LSTM,Dense,Input,Bidirectional
from nltk.tokenize.treebank import TreebankWordTokenizer
from scipy import spatial
from random import shuffle
import pickle as pk
from keras.callbacks import ModelCheckpoint
from scipy import spatial
import csv
import requests
import json
from textblob import TextBlob
from __future__ import print_function
import os
import sys
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

In [0]:
!apt-get install axel

In [0]:
!axel -n 30 http://nlp.stanford.edu/data/glove.6B.zip

In [0]:
!apt-get install unzip
!unzip glove.6B.zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.
Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
BASE_DIR = ''
GLOVE_DIR = os.path.join(BASE_DIR, '')
TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
batch_size = 32

In [0]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
df = df[:3000]

In [0]:
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

In [0]:
texts = list(df[0])

In [0]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
print(texts[0])
sequences = tokenizer.texts_to_sequences(texts)

a stirring , funny and finally transporting re imagining of beauty and the beast and 1930s horror films


In [0]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9046 unique tokens.


In [0]:
max_len = 0
for i in sequences:
  if len(i) > max_len:
    max_len = len(i)
MAX_SEQUENCE_LENGTH = max_len

In [0]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

numpy.ndarray

In [0]:
labels = np.array(df[1])

3000

In [0]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [0]:
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

In [0]:
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [0]:
embedding_layer

In [0]:
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
 
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=4,
          validation_data=[x_val, y_val])

Train on 2400 samples, validate on 600 samples
Epoch 1/4
2400/2400 [==============================] - 22s 9ms/step - loss: 0.2912 - acc: 0.8721 - val_loss: 0.4603 - val_acc: 0.7900
Epoch 2/4
2400/2400 [==============================] - 22s 9ms/step - loss: 0.2728 - acc: 0.8867 - val_loss: 0.5239 - val_acc: 0.8000
Epoch 3/4
2400/2400 [==============================] - 22s 9ms/step - loss: 0.2138 - acc: 0.9125 - val_loss: 0.6249 - val_acc: 0.7867
Epoch 4/4
2400/2400 [==============================] - 22s 9ms/step - loss: 0.1616 - acc: 0.9400 - val_loss: 0.6135 - val_acc: 0.7800
